# Embarrasingly parallel date operations

each row is processed independently

I want to calculate the number of days to and after the next holiday. As I am new to python I am unsure how to perform such a calculation efficiently

In [1]:
%load_ext Cython

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import datetime as DT

import matplotlib.pyplot as plt
plt.style.use('ggplot')
import time

load dates and holidays

In [3]:
datesFrame = pd.read_csv('myDates.csv')
datesFrame.myDates = pd.to_datetime(datesFrame.myDates)

holidays = pd.read_csv('holidays.csv')
holidays.day = pd.to_datetime(holidays.day)
holidays.type = holidays.type.astype("category")
holidays.name = holidays.name.astype("category")

holidays = holidays[holidays.apply(lambda x: (x.type == 'National holiday'), axis=1)]

In [4]:
def get_nearest_date(dates, pivot):
    nearest = min(dates, key=lambda x: abs(x - pivot))
    difference = abs(nearest - pivot)
    differenceAsDay = difference / np.timedelta64(1, 'D')
    return differenceAsDay.astype(int)

## approach 3 - parallelApply

Takes 4 min 18 seconds

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

def get_nearest_dateParallel(df):
    df['daysBeforeHoliday'] = df.myDates.apply(lambda x: get_nearest_date(holidays.day[holidays.day < x], x))
    df['daysAfterHoliday']  =  df.myDates.apply(lambda x: get_nearest_date(holidays.day[holidays.day > x], x))
    return df

def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)

print ('parallel version: ')
# 2 min 30 seconds
%time result = applyParallel(datesFrame.groupby(datesFrame.index), get_nearest_dateParallel)

parallel version: 


In [11]:
#result

## approach 5 sorted 
http://stackoverflow.com/questions/39284989/parallelize-pandas-apply?noredirect=1#comment65962782_39284989

Further improvement: How can I utilize pandas.tseries.holiday import USFederalHolidayCalendar for other national calendars instead of my CSV?

In [12]:
holidays = holidays.set_index('day')
holidays.index
holidays = holidays.drop(['name','type'], axis=1)

In [13]:
holidays.index

DatetimeIndex(['2013-01-01', '2013-01-06', '2013-04-01', '2013-05-01',
               '2013-05-09', '2013-05-20', '2013-05-30', '2013-08-15',
               '2013-10-26', '2013-11-01', '2013-12-08', '2013-12-25',
               '2013-12-26', '2014-01-01', '2014-01-06', '2014-04-21',
               '2014-05-01', '2014-05-29', '2014-06-09', '2014-06-19',
               '2014-08-15', '2014-10-26', '2014-11-01', '2014-12-08',
               '2014-12-25', '2014-12-26', '2015-01-01', '2015-01-06',
               '2015-04-06', '2015-05-01', '2015-05-14', '2015-05-25',
               '2015-06-04', '2015-08-15', '2015-10-26', '2015-11-01',
               '2015-12-08', '2015-12-25', '2015-12-26', '2016-01-01',
               '2016-01-06', '2016-03-28', '2016-05-01', '2016-05-05',
               '2016-05-16', '2016-05-26', '2016-08-15', '2016-10-26',
               '2016-11-01', '2016-12-08', '2016-12-25', '2016-12-26',
               '2017-01-01', '2017-01-06', '2017-04-17', '2017-05-01',
      

In [14]:
# datesFrame.myDates

All dates already are a timestamp -> why does it not work to convert them?

In [16]:
print(datesFrame.myDates.dtype)
print(holidays.index.dtype)
# holidays.index.astype(np.int64)

datetime64[ns]
datetime64[ns]


In [22]:
indices = holidays.index.astype(np.int64).searchsorted(datesFrame.myDates)
# array([1, 6, 9, 3])
next_nearest = holidays[indices]

TypeError: invalid type promotion

In [4]:
next_nearest

DatetimeIndex(['2016-01-18', '2016-10-10', '2016-12-26', '2016-05-30'], dtype='datetime64[ns]', freq=None)

In [5]:
next_nearest_diff = pd.to_timedelta(next_nearest.values - dates.values).days

In [6]:
next_nearest_diff

array([15, 31, 14, 88])

In [11]:
#from datetime import date
#from workalendar.europe import Germany
#cal = Germany()
#cal.holidays(2012)
#[(datetime.date(2012, 1, 1), 'New year'),
# (datetime.date(2012, 4, 9), 'Easter Monday'),
# (datetime.date(2012, 5, 1), 'Labour Day'),
# (datetime.date(2012, 5, 8), 'Victory in Europe Day'),
# (datetime.date(2012, 5, 17), 'Ascension Day'),
# (datetime.date(2012, 5, 28), 'Whit Monday'),
# (datetime.date(2012, 7, 14), 'Bastille Day'),
# (datetime.date(2012, 8, 15), 'Assumption of Mary to Heaven'),
# (datetime.date(2012, 11, 1), "All Saints' Day"),
# (datetime.date(2012, 11, 11), 'Armistice Day'),
# (datetime.date(2012, 12, 25), 'Christmas')]
#cal.is_working_day(date(2012, 12, 25))  # it's Christmas
#cal.is_working_day(date(2012, 12, 30))  # it's Sunday
#cal.is_working_day(date(2012, 12, 26))
#cal.add_working_days(date(2012, 12, 23), 5)  # 5 working days after Xmas
#datetime.date(2012, 12, 31)